### ResNet18: Tensorflow Pre-Trained

Due to difficulty overcoming overfitting with the previous approaches, and probably the data preprocessing being a potential source of error, 

the whole data importing and preprocessing was revised by first constructing a model with PyTorch and then translating it into tensorflow/keras

### Initialize Notebook & packages

In [1]:
import helper as hp 
hp.initialize_notebook() # initialize with GPU enabled  
# hp.initialize_notebook(False) # to disable GPU 

2024-12-15 21:19:32.962124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 21:19:32.985326: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 21:19:32.995121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 21:19:33.082027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU enabled. Checking for available GPUs...


I0000 00:00:1734297579.189522  681772 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


1 Physical GPUs, 1 Logical GPUs

Verifying TensorFlow and PyTorch CUDA setup...
TensorFlow version: 2.17.0
Built with CUDA: True
Num GPUs Available: 1

Keras version: 3.6.0

End checks and initialization.


I0000 00:00:1734297579.568320  681772 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734297579.572559  681772 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734297579.578241  681772 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734297579.581671  681772 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
# Import standard libraries
import os
import sys
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import keras
# Import DL libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import layers, models, Model, Input
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.callbacks import TensorBoard 

# Suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import ML libraries
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

# Import image libraries
from skimage import transform
import tifffile as tiff

# Appends current working dir
current_path = os.getcwd()
sys.path.append(current_path)

# Import custom preprocessing class
from imc_preprocessing import IMCPreprocessor

# Import Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd
import numpy as np
import tensorflow as tf
import tifffile as tiff
import os
from sklearn.model_selection import train_test_split


## IMPORT DATA

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tifffile import imread
images_dir = '/home/jupyter-luvogt/Final_Project_LR/IMC_images' 
metadata_dir = '/home/jupyter-luvogt/Final_Project_LR/metadata.csv'
# --- Load CSV and filter dataset ---
csv_file = metadata_dir
image_folder = images_dir

# Load the CSV
df = pd.read_csv(csv_file)

# Filter rows with NA in PDL1_score and convert to binary
df = df.dropna(subset=["PDL1_score"])
df["PDL1_score"] = df["PDL1_score"].astype(int)

# --- Train-Test-Validation Split ---
train_df, val_and_test_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df["PDL1_score"])
test_df, val_df = train_test_split(val_and_test_df, test_size=0.5, random_state=42, stratify=val_and_test_df["PDL1_score"])

# --- Load Images and Compute Mean and Standard Deviation ---

# Function to load a single image
def load_image(image_path):
    image = imread(image_path)  # Load all 46 channels
    return image.astype(np.float32)

# Initialize variables to accumulate sum and sum of squares
nr_images = 0
sum_images = np.zeros((46, 224, 224))
sum_squared_images = np.zeros((46, 224, 224))

# Accumulate the sum and sum of squares for the training dataset
for _, row in train_df.iterrows():
    image_path = os.path.join(image_folder, f"{row['sample_id']}.tiff")
    image = load_image(image_path)
    nr_images += 1
    sum_images += image
    sum_squared_images += image ** 2

# Compute mean and standard deviation
mean = sum_images / nr_images
std = np.sqrt(sum_squared_images / nr_images - mean ** 2)

# --- Dataset Loading Function ---

def load_dataset(dataframe, image_folder, augment=False):
    images = []
    labels = []
    
    for _, row in dataframe.iterrows():
        image_path = os.path.join(image_folder, f"{row['sample_id']}.tiff")
        image = load_image(image_path)

        # Normalize the image
        image = (image - mean) / std

        # Convert to channels-last format (224, 224, 46)
        image = np.transpose(image, (1, 2, 0))

        # Apply data augmentation if specified
        if augment:
            # Random horizontal flip
            if np.random.rand() > 0.5:
                image = np.flip(image, axis=2)
            # Random vertical flip
            if np.random.rand() > 0.5:
                image = np.flip(image, axis=1)

        images.append(image)
        labels.append(row["PDL1_score"])

    return np.array(images), np.array(labels)

# --- Create Datasets ---

# Training dataset without augmentation
X_train, y_train = load_dataset(train_df, image_folder, augment=False)

# Training dataset with data augmentation
X_train_aug, y_train_aug = load_dataset(train_df, image_folder, augment=True)

# Validation dataset
X_val, y_val = load_dataset(val_df, image_folder, augment=False)

# Test dataset
X_test, y_test = load_dataset(test_df, image_folder, augment=False)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (590, 224, 224, 46), y_train shape: (590,)
X_val shape: (197, 224, 224, 46), y_val shape: (197,)
X_test shape: (197, 224, 224, 46), y_test shape: (197,)


Verifying Normalization of data

In [ ]:
print(np.mean(X_train)) # Mean = 0
print(np.std(X_train)) # Standard deviation = 1

#### One-Hot Encoding

In [4]:
y_train= to_categorical(y_train)
y_val= to_categorical(y_val)
y_test = to_categorical(y_test)

In [6]:
from keras.regularizers import l2
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
# from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def ResNet(shape = (32, 32, 3), classes = 10, block_layers = [3, 4, 6, 3]):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = block_layers
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [7]:
# ROWS = 224
# COLS = 224
# CHANNELS = 46
# CLASSES = 2
# block_layers = [2,2,2,2]
# # Build Network Graph 
# model_ResNet18 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)

# # Compile Model 
# l_rate = 1.e-4
# opt = keras.optimizers.Adam(learning_rate=l_rate)

# model_ResNet18.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# # # Apply TensorBoard
# # # define the logs folder 
# # log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet50_3Channels" + time.strftime("%Y%m%d-%H%M%S"))
# # # Define TensorBoard Callback
# # tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# # Train Model 
# batch = 32
# epochs = 10
# start_time = time.time()

# history_ResNet18 = model_ResNet18.fit(X_train, y_train,
#                                       epochs = epochs, batch_size = batch, 
#                                       validation_data = (X_val, y_val))

# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"\nElapsed time: {elapsed_time} seconds")

In [8]:
# model_ResNet18.evaluate(X_test, y_test)

Finally, we have a Validation Accuracy above the baseline accuracy, giving us a proof of concept and giving us the ability to experiment further with: 

For now, we have 68.58% test accuracy

    - Different ResNet Architecture: ResNet18, ResNet50, and pre-trained ResNets
    - Data Augmentation
    - Dimensionality Reduction: Autoencoder
    - Feature Engineering: Channel Selection
    - Regularizations: Regularizer and Dropouts
    - Callbacks
    - Monitor hyperparameters with TensorBoard

## ResNet18: with Tensor Board + 46 Channels + without Regularization + no callbacks

In [11]:
from keras.callbacks import TensorBoard 
ROWS = 224
COLS = 224
CHANNELS = 46
CLASSES = 2
block_layers = [2,2,2,2] # ResNet18
# Build Network Graph 
model_ResNet18 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)

# Compile Model 
l_rate = 1.e-4
opt = keras.optimizers.Adam(learning_rate=l_rate)

model_ResNet18.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Apply TensorBoard
# define the logs folder 
log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet18_46_no_regularization_trained" + time.strftime("%Y%m%d-%H%M%S"))
# Define TensorBoard Callback
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train Model 
batch = 64
epochs = 10
start_time = time.time()

history_ResNet18 = model_ResNet18.fit(X_train, y_train,
                                      epochs = epochs, batch_size = batch, 
                                      validation_data = (X_val, y_val), callbacks = tb_callback)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

2024-12-15 21:02:24.733828: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5447106560 exceeds 10% of free system memory.
2024-12-15 21:02:30.556470: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5447106560 exceeds 10% of free system memory.


Epoch 1/10


2024-12-15 21:02:44.753749: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.12GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


10/10 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.5743 - loss: 1.5904 - val_accuracy: 0.6396 - val_loss: 1.6029
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 574ms/step - accuracy: 0.7853 - loss: 0.5136 - val_accuracy: 0.6447 - val_loss: 0.9970
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 548ms/step - accuracy: 0.8780 - loss: 0.2858 - val_accuracy: 0.4010 - val_loss: 1.5503
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 555ms/step - accuracy: 0.9531 - loss: 0.1533 - val_accuracy: 0.6802 - val_loss: 0.7513
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 561ms/step - accuracy: 0.9848 - loss: 0.0689 - val_accuracy: 0.6701 - val_loss: 0.8414
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 562ms/step - accuracy: 0.9822 - loss: 0.0776 - val_accuracy: 0.6396 - val_loss: 1.0069
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 541ms/step - accuracy: 0.9925 - loss: 0.0457 - val_accuracy: 0.7157 - val_loss: 0.8810
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 573ms/step - accuracy: 1.0000 - loss: 0.0152 - val_accuracy: 0.6853 - val_lo

In [12]:
#### Test accuracy
print("Test Accuracy is: {:.2f}%".format(model_ResNet18.evaluate(X_test, y_test)[1]))

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.6562 - loss: 1.0567
Test Accuracy is: 0.65%


#### SAVING MODEL

In [26]:
import pickle
# Save the model
model_ResNet18.save('./models_ResNet/model_ResNet18_46_no_regularization_trained.keras')
# Save the history of your experiments 
with open('./models_ResNet/history_red_LR.pkl', 'wb') as f:
    pickle.dump(history_ResNet18.history, f)

## ResNet18: with Tensor Board + 46 Channels + without Regularization + ReduceLR + EarlyStopping

In [14]:
# Callbacks

log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet18_46_no_regularization_trained" + time.strftime("%Y%m%d-%H%M%S"))
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=12,   # Optimal patience value for validation accuracy
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="./models_ResNet/model_ResNet18_46_no_regularization_trained.keras", # save validation loss into file
        monitor="val_loss",  # monitor validation accuracy
        save_best_only=True,
    ),
    ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,  
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0.001,
    min_lr=1.e-6,
    ), 
    TensorBoard(log_dir=log_dir, histogram_freq=1)
]

In [15]:
ROWS = 224
COLS = 224
CHANNELS = 46
CLASSES = 2
block_layers = [2,2,2,2] # ResNet18
# Build Network Graph 
model_ResNet18 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers)

# Compile Model 
l_rate = 1.e-4
opt = keras.optimizers.Adam(learning_rate=l_rate)

model_ResNet18.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Apply TensorBoard
# define the logs folder 
log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet18_46_no_regularization_trained" + time.strftime("%Y%m%d-%H%M%S"))
# Define TensorBoard Callback
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train Model 
batch = 64
epochs = 10
start_time = time.time()

history_ResNet18 = model_ResNet18.fit(X_train, y_train,
                                      epochs = epochs, batch_size = batch, 
                                      validation_data = (X_val, y_val), callbacks = callbacks_list)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

2024-12-15 21:08:32.109724: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5447106560 exceeds 10% of free system memory.


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.6366 - loss: 1.5402 - val_accuracy: 0.3604 - val_loss: 2.5013 - learning_rate: 1.0000e-04
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 716ms/step - accuracy: 0.8172 - loss: 0.5057 - val_accuracy: 0.4721 - val_loss: 0.9120 - learning_rate: 1.0000e-04
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 584ms/step - accuracy: 0.9285 - loss: 0.2298 - val_accuracy: 0.4822 - val_loss: 1.1409 - learning_rate: 1.0000e-04
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 720ms/step - accuracy: 0.9554 - loss: 0.1310 - val_accuracy: 0.6802 - val_loss: 0.7067 - learning_rate: 1.0000e-04
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 591ms/step - accuracy: 0.9975 - loss: 0.0662 - val_accuracy: 0.6447 - val_loss: 0.7182 - learning_rate: 1.0000e-04
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 771ms/step - accuracy: 0.9940 - loss: 0.0332 - val_accuracy: 0.7056 - val_loss: 0.6775 - learning_rate: 1.0000e-04
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 580ms/step - accuracy: 0.

In [16]:
#### Test accuracy
print("Test Accuracy is: {:.2f}%".format(model_ResNet18.evaluate(X_test, y_test)[1]))

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.7067 - loss: 0.9014
Test Accuracy is: 0.71%


#### COMMENTS

Test accuracy improved a little bit

#### SAVING MODEL

In [17]:
import pickle
# Save the model
model_ResNet18.save('./models_ResNet/model_ResNet18_46_no_regularization_callbacks_trained.keras')
# Save the history of your experiments 
with open('./models_ResNet/history_ResNet18_46_no_regularization_callbacks_trained.pkl', 'wb') as f:
    pickle.dump(history_ResNet18.history, f)

## ResNet18: with Tensor Board + 46 Channels + with Dropout + ReduceLR + EarlyStopping

### Redefine Network with Dropout

In [5]:
def identity_block(x, filter, dropout_rate = 0.5):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = Dropout(dropout_rate)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def convolutional_block(x, filter, dropout_rate = 0.5):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = Dropout(dropout_rate)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def ResNet(shape = (32, 32, 3), classes = 10, block_layers = [3, 4, 6, 3], dropout_rate = 0.5):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = block_layers
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [6]:
# Callbacks

log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet18_46_dropout_callbacks_trained" + time.strftime("%Y%m%d-%H%M%S"))
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=12,   # Optimal patience value for validation accuracy
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="./models_ResNet/model_ResNet18_46_dropout_callbacks_trained.keras", # save validation loss into file
        monitor="val_loss",  # monitor validation accuracy
        save_best_only=True,
    ),
    ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,  
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0.001,
    min_lr=1.e-6,
    ), 
    TensorBoard(log_dir=log_dir, histogram_freq=1)
]

In [9]:
ROWS = 224
COLS = 224
CHANNELS = 46
CLASSES = 2
block_layers = [2,2,2,2] # ResNet18
dropout_rate = 0.5
# Build Network Graph 
model_ResNet18 = ResNet(shape = (ROWS, COLS, CHANNELS), classes = CLASSES, block_layers = block_layers, dropout_rate = dropout_rate)

# Compile Model 
l_rate = 1.e-4
opt = keras.optimizers.Adam(learning_rate=l_rate)

model_ResNet18.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Apply TensorBoard
# define the logs folder 
log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet18_46_dropout_callbacks_trained" + time.strftime("%Y%m%d-%H%M%S"))
# Define TensorBoard Callback
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train Model 
batch = 64
epochs = 30
start_time = time.time()

history_ResNet18 = model_ResNet18.fit(X_train, y_train,
                                      epochs = epochs, batch_size = batch, 
                                      validation_data = (X_val, y_val), callbacks = callbacks_list)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

2024-12-15 21:24:09.128733: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5447106560 exceeds 10% of free system memory.
2024-12-15 21:24:16.389368: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5447106560 exceeds 10% of free system memory.


Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.6218 - loss: 1.4648 - val_accuracy: 0.3655 - val_loss: 1.6273 - learning_rate: 1.0000e-04
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 628ms/step - accuracy: 0.6907 - loss: 0.8077 - val_accuracy: 0.3858 - val_loss: 1.1247 - learning_rate: 1.0000e-04
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 618ms/step - accuracy: 0.6596 - loss: 0.6948 - val_accuracy: 0.4213 - val_loss: 0.8410 - learning_rate: 1.0000e-04
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 627ms/step - accuracy: 0.6950 - loss: 0.6337 - val_accuracy: 0.4975 - val_loss: 0.7305 - learning_rate: 1.0000e-04
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 625ms/step - accuracy: 0.7115 - loss: 0.5754 - val_accuracy: 0.4924 - val_loss: 0.8391 - learning_rate: 1.0000e-04
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 615ms/step - accuracy: 0.8052 - loss: 0.4526 - val_accuracy: 0.5838 - val_loss: 0.7443 - learning_rate: 1.0000e-04
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0

In [8]:
#### Test accuracy
print("Test Accuracy is: {:.2f}%".format(model_ResNet18.evaluate(X_test, y_test)[1]))

7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.7751 - loss: 0.6151
Test Accuracy is: 0.78%


#### SAVING MODEL

In [ ]:
import pickle
# Save the model
model_ResNet18.save('./models_ResNet/model_ResNet18_46_dropout_callbacks_trained.keras')
# Save the history of your experiments 
with open('./models_ResNet/model_ResNet18_46_dropout_callbacks_trained.pkl', 'wb') as f:
    pickle.dump(history_ResNet18.history, f)